## Manual 버전 데이터셋 제작

해당 코드는 기존의 데이터셋(demo/small/large)에서 설정한 갯수의 데이터만을 랜덤하게 가져와 새로운 사이즈(manual)의 데이터셋을 생성합니다.

In [1]:
import os
import csv
import random
import pandas as pd
from tqdm import tqdm
from os import path
from dataclasses import dataclass

In [3]:
"""
경로가 폴더를 나타낼 경우 Dir, 파일일 경우 Path로 명명
"""

size = "manual"
original_size = "demo"

datasetDir = f"MIND/{size}"
datasetTrainDir = f"{datasetDir}/train"
datasetTestDir = f"{datasetDir}/test"

datasetOriginalDir = f"MIND/{original_size}"
datasetOriginalTrainDir = f"{datasetOriginalDir}/train"
datasetOriginalTestDir = f"{datasetOriginalDir}/test"

processedDataDir = "preprocessed_data"
preTrainDir = f"{processedDataDir}/{size}/train"
preTestDir = f"{processedDataDir}/{size}/test"

user2intPath = f"{preTrainDir}/user2int.tsv"
word2intPath = f"{preTrainDir}/word2int.tsv"
category2intPath = f"{preTrainDir}/category2int.tsv"

wordEmbeddingDir = "word_embeddings"
wordEmbeddingPath = f"{wordEmbeddingDir}/glove.840B.300d.txt"

os.makedirs(datasetTrainDir, exist_ok=True)
os.makedirs(datasetTestDir, exist_ok=True)
os.makedirs(datasetOriginalTrainDir, exist_ok=True)
os.makedirs(datasetOriginalTestDir, exist_ok=True)
os.makedirs(preTrainDir, exist_ok=True)
os.makedirs(preTestDir, exist_ok=True)

In [4]:
@dataclass
class Args:
    dataset_dir: str
    dataset_original_dir: str
    train_out_dir: str
    test_out_dir: str
    user2int_path: str
    split_test_size: float
    n_negative: int

args = Args(
    dataset_dir = datasetDir,
    dataset_original_dir = datasetOriginalDir,
    train_out_dir = preTrainDir,
    test_out_dir = preTestDir,
    user2int_path = f"{preTrainDir}/user2int.tsv",
    split_test_size = 0.1,
    n_negative = 4
)

"""
※중요: train_data_number는 2 이상으로 설정해야 합니다.
나중에 데이터 전처리 과정에서 train_test_split()을 통해 train 데이터셋을 train/val로 쪼개기 때문입니다.
"""
train_data_number = 3000
test_data_number = 2000
random.seed(1234)

def pick_random_integers(min: int, max: int, num: int):
    if num > (max - min + 1):
        raise ValueError("샘플 개수는 데이터의 총 개수보다 적거나 같아야 합니다.")
    elif num < 1:
        raise ValueError("num은 0보다 커야합니다.")
    return random.sample(range(min, max + 1), num)

### 1. manual Train/Test behaviors.tsv 생성

In [5]:
def generate_behaviors_dataset(
        original_path: str,
        out_path: str,
        data_number: int
    ):
    # 원본 데이터셋을 불러옵니다.
    with open(original_path, 'r') as original_behavior_file:
        original_behaviors = original_behavior_file.readlines()

    # 원본 데이터셋에서 뽑아올 샘플의 인덱스를 랜덤으로 생성합니다.
    indexes = pick_random_integers(0, len(original_behaviors) - 1, data_number)

    # 새로운 behaviors.tsv 파일을 생성합니다.
    with open(out_path, 'w', newline='') as behavior_out_file:
        behaviors_writer = csv.writer(behavior_out_file, delimiter='\t')
        for index in tqdm(indexes):
            behavior_data = original_behaviors[index]
            behaviors_writer.writerow(behavior_data.strip().split('\t'))

generate_behaviors_dataset(
    path.join(args.dataset_original_dir, "train", "behaviors.tsv"),
    path.join(args.dataset_dir, "train", "behaviors.tsv"),
    train_data_number
)
generate_behaviors_dataset(
    path.join(args.dataset_original_dir, "test", "behaviors.tsv"),
    path.join(args.dataset_dir, "test", "behaviors.tsv"),
    test_data_number
)

100%|██████████| 2000/2000 [00:00<00:00, 117670.44it/s]


### 2. manual Train/Test news.tsv 생성

In [7]:
def generate_news_dataset(
        behaviors_path: str,
        original_news_path: str,
        out_path: str
    ):
    # 생성한 behaviors.tsv 데이터셋을 불러옵니다.
    with open(behaviors_path, 'r') as original_behavior_file:
        behaviors = original_behavior_file.readlines()

    # 원본 news.tsv 데이터셋을 불러옵니다.
    news_columns = ["news_id", "category", "subcategory", "title", "abstract", "url", "title_entitles", "abstract_entities"]
    original_news_df = pd.read_csv(original_news_path, sep='\t', header=None, names=news_columns, encoding='utf-8', index_col="news_id")

    # manual/train/behaviors.tsv로 선별한 모든 샘플에 포함된 뉴스 목록을 news.tsv에 저장하기 위한 전처리 과정을 시작합니다.
    news_collection = set()
    for behavior_data in tqdm(behaviors):
        imp_id, user_id, time, history, impressions = behavior_data.strip().split('\t')
        # NewsID가 들어 있는 history, impressions에서 ID만 빼옵니다.
        history = history.split(' ')
        impressions = [s.split('-')[0] for s in impressions.split(' ')]
        # 중복을 제거합니다.
        if history[0] != '':
            news_collection.update(history)
        if impressions[0] != '':
            news_collection.update(impressions)

    # 중복 없이 뽑아낸 모든 뉴스ID의 데이터를 선별합니다.
    news_df = original_news_df.loc[list(news_collection)]

    # 선별한 데이터를 news.tsv에 저장합니다.
    news_df.to_csv(out_path, sep='\t', header=None, encoding='utf-8')
    return news_df

train_news_df = generate_news_dataset(
    path.join(args.dataset_dir, "train", "behaviors.tsv"),
    path.join(args.dataset_original_dir, "train", "news.tsv"),
    path.join(args.dataset_dir, "train", "news.tsv")
)
test_news_df = generate_news_dataset(
    path.join(args.dataset_dir, "test", "behaviors.tsv"),
    path.join(args.dataset_original_dir, "test", "news.tsv"),
    path.join(args.dataset_dir, "test", "news.tsv")
)

100%|██████████| 2000/2000 [00:00<00:00, 105266.82it/s]


## 기타

In [8]:
print(train_news_df.shape)
print(test_news_df.shape)

(17890, 7)
(12790, 7)
